# 多输入输出通道

## 多个输入通道
- 彩色图像可能有RGB三个通道
- 转为灰度会丢失信息

- 每个通道都有一个卷积核，结果是所有通道卷积结果的
- 
- 输入$X: c_i \times n_h \times n_w$
- 核$W: c_i\times k_h \times k_w$
- 输出$Y: m_h \times m_w$ $$Y=\sum_{i=0}{c_i}x_{i,:,:}*W_{i.:,:}$$

## 多个输出通道
- 无论有多少输入通道，到目前为止我们只用到单输出通道
- 我们可以有多个三维卷积核，每个核生成一个输出通道
- 输入$X: c_i \times n_h \times n_w$
- 核$W: c_o \times c_i\times k_h \times k_w$
- 输出$Y:c_o \times m_h \times m_w$ $$Y=\sum_{i=0}{c_i}x_{i,:,:}*W_{i.:,:}\quad for \ i=1,…,c_o$$

- 每个输出通道可以识别特定模式
- 输入通道核识别并组合输入中的模式

## $1\times 1卷积层$
- $k_h =k_w =1$是一个受欢迎的选择。他不识别空间模式，只是融合通道。
- 相当于输入形状为$n_h n_w \times c_i $，权重为$c_o \times c_i$的全连接

## 二维卷积层
- 输入$X: c_i \times n_h \times n_w$
- 核$W: c_o \times c_i\times k_h \times k_w$
- 偏差$B: c_o \times c_i$
- 输出$Y:c_o \times m_h \times m_w$ $$Y=X*W+B$$
- 计算复杂度：$O(c_i c_o k_h k_w m_h m_w)$

## 总结
- 输出通道数是卷积层的超参数
- 每个输入通道有独立的二维卷积核，所有通道结果相加得到一个输出通道结果
- 每个输出通道有独立的三维卷积核

# 代码实现

## 多输入多输出通道

多输入通道互相关运算

In [1]:
import torch
import d2l_source.d2l as d2l


def corr2d_multi_in(X, K):
    # 先遍历“X”和“K”的第0个维度（通道维度），再把它们加在一起
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

验证输出

In [2]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

计算多通道输出的互相关函数

In [3]:
def corr2d_multi_in_out(X, K):
    # 迭代“K”的第0个维度，每次都对输入“X”执行互相关运算。
    # 最后将所有结果都叠加在一起
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

K = torch.stack((K, K + 1, K + 2), 0)
K.shape

torch.Size([3, 2, 2, 2])

In [4]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

$1\times 1$卷积

In [5]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    # 全连接层中的矩阵乘法
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6